In [4]:
from time import sleep
import requests as rq
from tqdm import tqdm
import random
import re
from datetime import datetime as dt
import sys
sys.setrecursionlimit(100000)

# 1. Conseguir IDs de todos los papers

In [ ]:
fieldsOfStudy = [
    'Computer Science',
    'Medicine',
    'Chemistry',
    'Biology',
    'Materials Science',
    'Physics',
    'Geology',
    'Psychology',
    'Art',
    'History',
    'Geography',
    'Sociology',
    'Business',
    'Political Science',
    'Economics',
    'Philosophy',
    'Mathematics',
    'Engineering',
    'Environmental Science',
    'Agricultural and Food Sciences',
    'Education',
    'Law',
    'Linguistics'
]

years = [
    '-2010',
    '2011-2016',
    '2017-2021',
    '2022-'
]

def connection_test():
    try:
        rq.get("https://www.google.com", timeout=(5,15))
        return True
    except:
        return False

def request (url, delay=1, attempt=1, max_attempt=10, tol=60):
    # Asegura continuación ante saturaciones de red.
    try:
        sleep(delay)
        response = rq.get(url,timeout=(5,15))
        if response.status_code == 200:
            # Retorno de los resultados de la API.
            with open('logs/delaylog.txt','a') as f:
                f.write(f"{delay},{attempt},{url}\n")
                # TEMPORAL: Quiero analizar la saturación de requests al API en su rate limit de 1000 resultados.

            return response.json()
        elif response.status_code != 200 and attempt!=max_attempt:
            # Principalmente maneja errores 429 y 504.
            # Si yo supero el rate limit o el server no responde a tiempo, se aplica exponential backoff + reintento recursivo.
            delay = min(2 ** attempt + random.uniform(0,1), 1024)
            return request(url, delay, attempt=attempt+1, max_attempt=max_attempt)
        else:
            # Registro de error y devolución de bandera 'None'.
            with open('logs/erroridslog.txt','a') as f:
                f.write(f'ERROR {response.status_code} at attempt {attempt}: {url}\n')
            
            return None
        
    # Detiene la función hasta que se haya reconexión.
    except rq.exceptions.RequestException as e:
        with open('connectionlog.txt','a') as f:
            f.write(f'[{dt.now()} - {e}] Lost connection. Awaiting reconnection.')
        
        while not connection_test():
            sleep(tol)

        with open('connectionlog.txt','a') as f:
            f.write(f'[{dt.now()}] Reconnected.')

        return request(url,delay=delay,attempt=attempt,max_attempt=max_attempt,tol=tol)

def fetchIDs(field,year,token=None):
    url = f'https://api.semanticscholar.org/graph/v1/paper/search/bulk?query=*&fieldsOfStudy={field}&year={year}' + (f'&token={token}' if token!=None else '')
    response = request(url,max_attempt=20)
    
    if response is not None:
        if not token:
            # Solo la primera iteración de un nuevo fetch retornará resultados.
            print(f'field: {field}\nyears: {year}\ntotal results: {response['total']}\nestimated amount of paginations: {response['total']//1000 + 1}\nleast amount of processing hours: {(response['total']//1000 + 1)//3600 + 1}\n')
        
        # Registro de IDs
        with open('paperIds.txt','a') as f:
            f.write('\n'.join(re.findall(r"\{'paperId': '.+?', 'title': '.+?'\}",str(response['data']),re.DOTALL)) + '\n')
        
        # Encontrar nuevo token de paginación, si hay.
        next_token = response.get('token') 
        with open('logs/tokenlog.txt','a') as f:
                f.write(str(next_token)+'\n')
                # TEMPORAL: Quiero ver el comportamiento de los tokens.

        # Continuación recursiva.
        if next_token:
            fetchIDs(field,year,token=next_token)

Considerando que en total son 225'848'910 artículos y actualmente trabajo con un rate limit de ~1000 resultados en requests/segundo, para una sola IP en secuencia demorará aproximadamente 2 días y medio en recuperar todos los IDs.
 * Ponderar distribución de solicitudes en otras máquinas o en máquinas virtuales.
 * Ojalá tuviera ya la licencia autorizada para la API, que me permitiría en principio descargar todo el dataset en el endpoint de baches. Ello simplificaría el paso 1 y 2 en solo unas cuantas solicitudes.

In [ ]:
with tqdm(total=len(fieldsOfStudy)*len(years)) as pbar:
    for field in fieldsOfStudy:
        for year in years:
            fetchIDs(field,year)
            pbar.update(1)

# 2. Buscar metadata para cada ID

# 3. Buscar PDFs